In [2]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Set the option to prevent the FutureWarning
pd.set_option('future.no_silent_downcasting', True)
import datetime as dt
import os
os.environ['PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT'] = '2'

colTypes = {
    'route_id': 'string',
    'direction_id': 'category',
    'half_trip_id': 'string',
    'stop_id': pd.Int32Dtype(),
    'time_point_order': pd.Int8Dtype(),
    'point_type': 'category', 
    'standard_type': 'category'
}

In [3]:
# Import all *.txt files in the gtfsSchedule folder and parse them as dataframes
# add the gtfsSchedule folder and subfolders to the current path
txt_path = ('gtfsSchedule\\gtfs_2022-12-18_2023-03-11_Winter2023PostRecap')
# List of dataframe names: remove the '.txt' extension from the filenames
df_names = [filename[:-4] for filename in os.listdir(txt_path)] 
# Read txt files into dataframes and assign them the names in df_names
# create a list of filenames
files = os.listdir(txt_path)
# create a list of dataframes
dfs = [pd.read_csv(os.path.join(txt_path, f), sep=',') for f in files]
# create a dictionary of dataframes
gtfsSchedule = dict(zip(df_names, dfs))

C:\Users\antonio.forte\AppData\Local\Temp\ipykernel_16408\3995012852.py:10: DtypeWarning: Columns (0,3,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(os.path.join(txt_path, f), sep=',') for f in files]
C:\Users\antonio.forte\AppData\Local\Temp\ipykernel_16408\3995012852.py:10: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(os.path.join(txt_path, f), sep=',') for f in files]


In [23]:
# Create a new df starting from gtfsScehdule['trips] where trip_ids are matched to service_ids
# This will be the base df for the analysis
trips = gtfsSchedule['trips']
calendar = gtfsSchedule['calendar']

calendar_attributes = gtfsSchedule['calendar_attributes']
calendar_attributes_dtypes = {'service_id': 'string',
'service_description': 'string',
'service_schedule_name': 'string',
'service_schedule_type': 'string',
'service_schedule_typicality': pd.Int16Dtype(),
'rating_start_date': 'string',
'rating_end_date':'string',
'rating_description':'string'}

for key, value in calendar_attributes_dtypes.items():
    calendar_attributes[key] = calendar_attributes[key].astype(value)
# Remove the leading 0 from the rating_start_date and rating_end_date fields
calendar_attributes['rating_end_date'] = calendar_attributes['rating_end_date'].str.replace('.0', '', regex=False)

calendar_dates = gtfsSchedule['calendar_dates']

# Assign to every row in trips the corresponding rating_start_date and rating_end_date contained in calendar_attributes matching the two datasets by service_id
# Add the service_id field from the calendar dataframe to the trips dataframe, without including the other fields
trips = pd.merge(trips, calendar_attributes[['service_id', 'rating_start_date', 'rating_end_date']], on='service_id')

# Extract the stop_times df from the dict
stop_times = gtfsSchedule['stop_times']
stop_times.trip_id = stop_times.trip_id.astype('string')
# Merge stop_times with trips
schedule = pd.merge(stop_times, trips, on='trip_id', how='left')

schedule_cols = {
    'trip_id': 'string',
    'arrival_time': 'string',
    'departure_time': 'string',
    'stop_id': 'string',
    'stop_sequence': pd.Int16Dtype(),
    'timepoint': pd.Int16Dtype(),
    'checkpoint_id': 'category',
    'route_id': 'string',
    'service_id': 'string',
    'direction_id': 'category',
    'block_id': 'string',
    'rating_start_date': 'string',
    'rating_end_date': 'string'
}

for key, value in schedule_cols.items():
    schedule[key] = schedule[key].astype(value)

# Parse datetime strings to datetime objects
schedule["arrival_time"] = schedule["arrival_time"].str.replace(r'^24', '00', regex=True) \
                         .str.replace(r'^25', '01', regex=True) \
                         .str.replace(r'^26', '02', regex=True) \
                         .str.replace(r'^27', '03', regex=True) \
                         .str.replace(r'^28', '04', regex=True)
schedule['arrival_time'] = pd.to_datetime(schedule['arrival_time'], format='%H:%M:%S')
schedule["departure_time"] = schedule["departure_time"].str.replace(r'^24', '00', regex=True) \
                         .str.replace(r'^25', '01', regex=True) \
                         .str.replace(r'^26', '02', regex=True) \
                         .str.replace(r'^27', '03', regex=True) \
                         .str.replace(r'^28', '04', regex=True)
schedule['departure_time'] = pd.to_datetime(schedule['departure_time'], format='%H:%M:%S')
#schedule['arrival_time'] = schedule['arrival_time'].dt.strftime("%H:%M:%S")
#schedule['departure_time'] = schedule['departure_time'].dt.strftime("%H:%M:%S")


schedule['rating_start_date'] = pd.to_datetime(schedule['rating_start_date'], format='%Y%m%d')
schedule['rating_end_date'] = pd.to_datetime(schedule['rating_end_date'], format='%Y%m%d')

# Extract only records whose route_id is an integer, i.e., bus routes
schedule = schedule[schedule.route_id.str.isnumeric()]
# Drop the columns that are not needed for the analysis
drop_colums = ['trip_headsign', 'trip_short_name',
        'shape_id', 'wheelchair_accessible',
       'trip_route_type', 'route_pattern_id', 'bikes_allowed', 'stop_headsign',
       'pickup_type', 'drop_off_type', 
       'continuous_pickup', 'continuous_drop_off']
schedule = schedule.drop(columns=drop_colums)

In [35]:
schedule.loc[schedule.departure_time == '1900-01-01 08:00:00']

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,timepoint,checkpoint_id,route_id,service_id,direction_id,block_id,rating_start_date,rating_end_date
32832,54241723,1900-01-01 08:00:00,1900-01-01 08:00:00,2519,1,1,sprhl,85,WinterWeekday,1,T88-161,2022-12-18,2023-03-11
33524,54241768,1900-01-01 08:00:00,1900-01-01 08:00:00,7967,6,0,NaN,67,WinterWeekday,1,T67-20,2022-12-18,2023-03-11
34132,54241803,1900-01-01 08:00:00,1900-01-01 08:00:00,141,1,1,alewf,67,WinterWeekday,0,T67-18,2022-12-18,2023-03-11
34359,54241814,1900-01-01 08:00:00,1900-01-01 08:00:00,2623,16,0,NaN,87,WinterWeekday,0,T88-165,2022-12-18,2023-03-11
34990,54241836,1900-01-01 08:00:00,1900-01-01 08:00:00,2674,9,0,NaN,88,WinterWeekday,1,T88-163,2022-12-18,2023-03-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429216,55049622,1900-01-01 08:00:00,1900-01-01 08:00:00,109853,1,1,pshly,712,MartinLutherKingDay-1,1,P-S1,2022-12-18,2023-03-11
2430721,55049676,1900-01-01 08:00:00,1900-01-01 08:00:00,15880,1,1,orhgt,712,MartinLutherKingDay-1,0,P-S2,2022-12-18,2023-03-11
2431326,55049817,1900-01-01 08:00:00,1900-01-01 08:00:00,109853,1,1,pshly,713,ChristmasDay(Observed)-1,1,P-T1,2022-12-18,2023-03-11
2432863,55049984,1900-01-01 08:00:00,1900-01-01 08:00:00,109853,1,1,pshly,713,ChristmasDay-1,1,P-T1,2022-12-18,2023-03-11


### Import files with arrival and departure times

In [5]:
# Import csv files from 2022 and 2023, cast them into a single dataframe, and filter out the bus routes included within the dates of the scheduled df
# Import the csv files
csv_path = 'MBTA_ArrivalDepartureTimes'
foldername = 'MBTA_Bus_Arrival_Departure_Times'

adt2022_list = []
adt2023_list = []
import_process = 0
if import_process:
    # Arrival/Departure times 2022
    adt_2022 = os.path.join(csv_path, (foldername + '_' + '2022'))
    csv2022_files = os.listdir(adt_2022)
    # Arrival/Departure times 2023
    adt_2023 = os.path.join(csv_path, (foldername + '_' + '2023'))
    csv2023_files = os.listdir(adt_2023)
    
    for i in range(12):
        print(i)
        adt2022_list.append(pd.read_csv(os.path.join(adt_2022, csv2022_files[i]), sep=','))
        adt2023_list.append(pd.read_csv(os.path.join(adt_2023, csv2023_files[i]), sep=','))

    # Build a single dataframe
    adt_df = pd.concat((pd.concat(adt2023_list, axis = 0), pd.concat(adt2022_list, axis = 0)), axis = 0)
    # Keep only the rows whose service_date is within the range of the scheduled df
    feed_info = gtfsSchedule['feed_info']
    start_date = pd.to_datetime(feed_info.feed_start_date.values, format='%Y%m%d')
    end_date = pd.to_datetime(feed_info.feed_end_date.values, format='%Y%m%d')
    adt_df['service_date'] = pd.to_datetime(adt_df['service_date'], format='%Y-%m-%d')
    adt_df = adt_df.loc[adt_df.service_date >= start_date[0]]
    adt_df = adt_df.loc[adt_df.service_date <= end_date[0]]
    adt_df.reset_index()
    # Replace wrong SL3 id with the correct one
    adt_df.loc[adt_df.route_id=='746_', 'route_id'] = '746'
    # Use the routes gtfs file to match route_ids in the adt dataframe with their univocal identifier
    routes = gtfsSchedule['routes']
    adt_df = pd.merge(adt_df, routes[['route_id', 'route_short_name']], on='route_id')
    # Change the dtype of the columns included in the colTypes to their corresponding values
    for key, value in colTypes.items():
        adt_df[key] = adt_df[key].astype(value)
else:
    # Read the file in separate chunks and concatenate them
    chunk_size = 10**6
    chunks = []
    dtype_map = {
    "service_date": "string",
    "route_id": "string",
    "direction_id": "category",
    "half_trip_id": "string",
    "stop_id": "string",
    "time_point_id": "category", 
    "time_point_order": pd.Int16Dtype(),
    "point_type": "category", 
    "standard_type": "category",  
    "scheduled": "string",  # Consider converting to datetime later
    "actual": "string",  # Consider converting to datetime later
    "scheduled_headway": pd.Int32Dtype(),
    "headway": pd.Int32Dtype(),
    "route_short_name": "category"
    }
    for chunk in pd.read_csv('adt_df.csv', dtype=dtype_map, chunksize=chunk_size):
        chunks.append(chunk)
    adt_df = pd.concat(chunks, axis=0, ignore_index=True)
    # Convert service_date, scheduled and actual columns to datetime objects
    adt_df['scheduled'] = pd.to_datetime(adt_df['scheduled'], format='ISO8601')
    adt_df['actual'] = pd.to_datetime(adt_df['actual'], format='ISO8601')
    adt_df["scheduled"] = adt_df["scheduled"].dt.strftime("%H:%M:%S")
    adt_df["actual"] = adt_df["actual"].dt.strftime("%H:%M:%S")
    # If half_trip_id endswith '.0', trim this piece
    adt_df['half_trip_id'] = adt_df['half_trip_id'].str.replace('.0', '', regex=False)


### Combine arrival and departure times with scheduled information

In [6]:
# Let's first try a test on a subset of the data, with route_id == 10
# From the folder, import calendar_csv
calendar_csv_path = os.path.join('CalendarDates', 'calendar_df.csv')
calendar_df = pd.read_csv(calendar_csv_path, sep=',')
# Convert the date column to datetime
calendar_df['date'] = pd.to_datetime(calendar_df['date'], format='%Y-%m-%d')
# Convert day_of_week to category
calendar_df['day_of_week'] = calendar_df['day_of_week'].astype('category')
# Convert service_ids to dictionary
calendar_df['service_ids'] = calendar_df['service_ids'].apply(eval)
calendar_df.head()

,date,day_of_week,service_ids
0,2022-12-18,Sunday,"{FLL 22-S-Su-1-Sb4, FLL 22-N-Su-1-S72, FLL 22-..."
1,2022-12-19,Monday,"{Fall/WinterWeekday, RTL123-A-Wdy-01, RTL123-D..."
2,2022-12-20,Tuesday,"{Fall/WinterWeekday, RTL123-A-Wdy-01, RTL123-D..."
3,2022-12-21,Wednesday,"{Fall/WinterWeekday, RTL123-A-Wdy-01, RTL123-D..."
4,2022-12-22,Thursday,"{Fall/WinterWeekday, RTL123-A-Wdy-01, RTL123-D..."


In [42]:
# Create a copy of the adt_df not to mess with the original
adt_route10 = adt_df.copy()
# Extract records with route_id == 10 and time_point_order == 1
route10_mask = (adt_route10.route_id == '10') & (adt_route10.time_point_order == 1)
adt_route10 = adt_route10.loc[route10_mask]
# If there are any nan values in the actual columns, replace them with the scheduled values
adt_route10['actual'] = adt_route10['actual'].fillna(adt_route10['scheduled'])
# Drop the columns that are not needed for the analysis
drop_columns = ['point_type', 'scheduled_headway', 'headway', 'route_short_name']
adt_route10 = adt_route10.drop(columns=drop_columns)
adt_route10 = adt_route10.reset_index(drop=True)
# Rename actual to departure_time
adt_route10 = adt_route10.rename(columns={'scheduled': 'departure_time'})
# Replace 'Inbound' entries with 1 and 'Outbound' entries with 0
adt_route10['direction_id'] = adt_route10['direction_id'].cat.rename_categories({'Inbound': 1, 'Outbound': 0})
# Convert service_date, scheduled and departure_time to datetime objects
adt_route10['service_date'] = pd.to_datetime(adt_route10['service_date'], format='%Y-%m-%d')
adt_route10['departure_time'] = pd.to_datetime(adt_route10['departure_time'], format='%H:%M:%S')
adt_route10['actual'] = pd.to_datetime(adt_route10['actual'], format='%H:%M:%S')
# Add a new service_id column to the adt_route10 dataframe made of empty sets
adt_route10['service_id'] =''

In [47]:
# Merge adt_route10 and schedule on the route_id and departure_time columns
schedule_route10 = schedule.copy()
# Extract only the records with route_id == 10 and stop_sequence == 1
schedule_route10 = schedule_route10.loc[(schedule_route10.route_id == '10') & (schedule_route10.stop_sequence == 1)]

grouping_var = ['route_id', 'direction_id', 'departure_time'] #these are the same for both adt_route10 and schedule_route10
# Start looping across groups obtained from schedule_route10
schedule_route10_grouped = schedule_route10.groupby(grouping_var, observed=True)
adt_route10_grouped = adt_route10.groupby(grouping_var, observed=True)
for name, group in adt_route10_grouped:
    print(name)
    # print the group if an error occurs
    if name in schedule_route10_grouped.groups:
        print(group)
        # extract the corresponding group from schedule_route10_grouped
        schedule_group = schedule_route10_grouped.get_group(name)
        schedule_services = set(schedule_group['service_id'])

        # extract the subset of the calendar_df that matches the service_date
        service_days = calendar_df.loc[calendar_df.date.isin(group.service_date)]
        
        # loop through the service_days
        for i, row in service_days.iterrows():
            # add the intersection between schedule_services and row['service_ids'] to the service_id column in adt_route10 as plain strings
            adt_service_ids = schedule_services.intersection(row['service_ids'])
            adt_service_ids_str = ', '.join(adt_service_ids)  # Convert set to string
            adt_route10.loc[group.index, 'service_id'] = adt_service_ids_str
    else:
        print(name)
        #print(group)

('10', 1, Timestamp('1900-01-01 00:15:00'))
     service_date route_id direction_id half_trip_id stop_id time_point_id  \
24     2023-01-01       10            1     58062094      33         ctypt   
74     2023-01-02       10            1     58068021      33         ctypt   
521    2023-01-08       10            1     58131748      33         ctypt   
1046   2023-01-15       10            1     58208469      33         ctypt   
1556   2023-01-22       10            1     58281769      33         ctypt   
1993   2023-01-29       10            1     58346161      33         ctypt   
2602   2023-02-05       10            1     58421809      33         ctypt   
3121   2023-02-12       10            1     58498577      33         ctypt   
3643   2023-02-19       10            1     58574656      33         ctypt   
4143   2023-02-26       10            1     58647085      33         ctypt   
4673   2023-03-05       10            1     58723379      33         ctypt   
5199   2022-12-18   

In [43]:
# let me show rows of adt route 10 with a service_id that is not empty
adt_route10.time_point_order.unique()

<IntegerArray>
[1]
Length: 1, dtype: Int16